In [ ]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import regexp_replace, col
import json
import base64

class KafkaStreamConsumer:
    def __init__(self, topic, bootstrap_servers, group_id, hdfs_path, max_records):
        self.topic = topic
        self.bootstrap_servers = bootstrap_servers
        self.group_id = group_id
        self.hdfs_path = hdfs_path
        self.max_records = max_records
        self.record_count = 0

        self.consumer = KafkaConsumer(
            self.topic,
            bootstrap_servers=self.bootstrap_servers,
            group_id=self.group_id,
            value_deserializer=lambda x: json.loads(x.decode('utf-8'))
        )

        # Initialize Spark session
        self.spark = SparkSession.builder \
            .appName("KafkaToSpark") \
            .getOrCreate()
        
        # Define schema as StructType
        self.schema = StructType([
            StructField("Price", StringType(), True),
            StructField("Shape", StringType(), True),
            StructField("Carat", StringType(), True),
            StructField("Clarity", StringType(), True),
            StructField("Color", StringType(), True),
            StructField("Cut", StringType(), True),
            StructField("Polish", StringType(), True),
            StructField("Symmetry", StringType(), True),
            StructField("Fluorescence", StringType(), True),
            StructField("Certificate No", StringType(), True),
            StructField("Lab", StringType(), True),
            StructField("View", StringType(), True)
        ])

    def preprocess_data(self, data):
        print(f"Preprocessing data: {data}")  # Debug print
        
        # Base64 decode the data
        base64_str = data.get('$binary', {}).get('base64', '')
        if base64_str:
            decoded_bytes = base64.b64decode(base64_str)
            decoded_str = decoded_bytes.decode('utf-8')
            data = json.loads(decoded_str)
        
        # Convert the dictionary to a Spark DataFrame
        df = self.spark.createDataFrame([data], schema=self.schema)
        
        # Perform preprocessing
        df = df.withColumn("Price", regexp_replace(col("Price"), "RM|,", "")) \
               .withColumn('Price', col('Price').cast(DoubleType())) \
               .withColumn('Carat', col('Carat').cast(DoubleType())) \
               .drop('Cut', 'Lab', 'View', 'Certificate No')

        # Print DataFrame schema and data
        df.printSchema()
        df.show(truncate=False)
        
        return df

    def start_consuming(self):
        print("Starting to consume messages from Kafka...")
        try:
            for message in self.consumer:
                if self.record_count >= self.max_records:
                    print(f"Reached {self.max_records} records. Stopping consumption.")
                    break
                preprocessed_df = self.preprocess_data(message.value)
                if not preprocessed_df.rdd.isEmpty():  # Check if DataFrame is empty
                    self.process_message(preprocessed_df)
                else:
                    print("Empty DataFrame, skipping...")
                self.record_count += 1
                print(f"Processed record number: {self.record_count}")
        except KeyboardInterrupt:
            print("Consumption stopped by user.")
        except Exception as e:
            print(f"Error during consumption: {e}")
        finally:
            self.cleanup()

    def process_message(self, df):
        # Save DataFrame to HDFS
        self.save_to_file(df)

    def save_to_file(self, df):
        try:
            # Save DataFrame to JSON file in HDFS
            df.write.mode('append').json(self.hdfs_path)
            print("Data saved to file")
        except Exception as e:
            print(f"Error saving data to HDFS: {e}")

    def cleanup(self):
        self.consumer.close()
        self.spark.stop()
        print("Consumer and Spark session closed.")

if __name__ == "__main__":
    kafka_topic = 'diamond_csv'
    kafka_bootstrap_servers = ['localhost:9092']
    kafka_group_id = 'my-group'
    hdfs_path = 'hdfs://localhost:9000/user/student/de-dir'  # Specify your HDFS path here
    max_records = 1907  # Number of records to process

    consumer = KafkaStreamConsumer(
        topic=kafka_topic,
        bootstrap_servers=kafka_bootstrap_servers,
        group_id=kafka_group_id,
        hdfs_path=hdfs_path,
        max_records=max_records
    )

    consumer.start_consuming()


24/09/09 01:54:58 WARN Utils: Your hostname, LAPTOP-71BVR84G. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/09 01:54:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 01:54:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Starting to consume messages from Kafka...
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7bd7', 'Price': 'RM 3,734.00', 'Shape': 'RD', 'Carat': '0.30', 'Clarity': 'IF', 'Color': 'D', 'Cut': 'EX', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******0565', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)



+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|3734.0|RD   |0.3  |IF     |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+



Data saved to file
Processed record number: 1
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7bd8', 'Price': 'RM 4,126.00', 'Shape': 'RD', 'Carat': '0.33', 'Clarity': 'IF', 'Color': 'D', 'Cut': 'EX', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******6590', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|4126.0|RD   |0.33 |IF     |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+

Data saved to file
Processed reco

Data saved to file
Processed record number: 41
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7c00', 'Price': 'RM 3,484.00', 'Shape': 'RD', 'Carat': '0.40', 'Clarity': 'VVS1', 'Color': 'E', 'Cut': 'EX', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'F', 'Certificate No': '******2586', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|3484.0|RD   |0.4  |VVS1   |E    |EX    |EX      |F           |
+------+-----+-----+-------+-----+------+--------+------------+

Data saved to file
Processed r

...

Data saved to file
Processed record number: 793
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7cfc', 'Price': 'RM 9,517.00', 'Shape': 'RD', 'Carat': '0.51', 'Clarity': 'VVS1', 'Color': 'D', 'Cut': 'EX+', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******5692', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|9517.0|RD   |0.51 |VVS1   |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+

Data saved to file
Processed

Data saved to file
Processed record number: 795
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7cfe', 'Price': 'RM 9,615.00', 'Shape': 'RD', 'Carat': '0.51', 'Clarity': 'VVS1', 'Color': 'D', 'Cut': 'EX+', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******3289', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|9615.0|RD   |0.51 |VVS1   |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+



Data saved to file
Processed record number: 796
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7cff', 'Price': 'RM 9,125.00', 'Shape': 'RD', 'Carat': '0.52', 'Clarity': 'VVS1', 'Color': 'D', 'Cut': 'EX+', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******7887', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|9125.0|RD   |0.52 |VVS1   |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+



Data saved to file
Processed record number: 797
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7d00', 'Price': 'RM 9,445.00', 'Shape': 'RD', 'Carat': '0.54', 'Clarity': 'VVS1', 'Color': 'D', 'Cut': 'EX+', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******4910', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+------+-----+-----+-------+-----+------+--------+------------+
|Price |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+------+-----+-----+-------+-----+------+--------+------------+
|9445.0|RD   |0.54 |VVS1   |D    |EX    |EX      |N           |
+------+-----+-----+-------+-----+------+--------+------------+

Data saved to file
Processed

Data saved to file
Processed record number: 802
Preprocessing data: {'_id': '66dd8b0912d1dc09f57d7d05', 'Price': 'RM 10,134.00', 'Shape': 'RD', 'Carat': '0.56', 'Clarity': 'VVS1', 'Color': 'D', 'Cut': 'EX+', 'Polish': 'EX', 'Symmetry': 'EX', 'Fluorescence': 'N', 'Certificate No': '******3525', 'Lab': 'GIA', 'View': 'More Information'}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|10134.0|RD   |0.56 |VVS1   |D    |EX    |EX      |N           |
+-------+-----+-----+-------+-----+------+--------+------------+

Data saved to file
Pro